# Load libraries

In [1]:
import sqlalchemy
sqlalchemy.__version__

'1.4.39'

# Create SQLite engine
- more about connecting to databases here: https://docs.sqlalchemy.org/en/14/core/engines.html#database-urls

In [3]:
import sqlalchemy as db

engine = db.create_engine('sqlite:///european_database.sqlite')
conn = engine.connect()

In [6]:
metadata = db.MetaData()   # extracting the metadata
division = db.Table('divisions', metadata, autoload=True, autoload_with=engine)   # Table object

In [7]:
print(repr(metadata.tables['divisions']))

Table('divisions', MetaData(), Column('division', TEXT(), table=<divisions>), Column('name', TEXT(), table=<divisions>), Column('country', TEXT(), table=<divisions>), schema=None)


In [8]:
# print column names:
print(division.columns.keys())

['division', 'name', 'country']


## Simple SQL query

In [9]:
query = division.select()  # ==> SELECT * FROM divisions
print(query)

SELECT divisions.division, divisions.name, divisions.country 
FROM divisions


In [12]:
# same as above
print(db.select([division]))

SELECT divisions.division, divisions.name, divisions.country 
FROM divisions


## SQL query result

We will now execute the query using the connection object and extract the first five rows. 
- fetchone(): it will extract a single row at a time.
- fetchmany(n): it will extract the n number of rows at a time.
- fetchall(): it will extract all of the rows.  


In [13]:
exe = conn.execute(query)   # executing the query
result = exe.fetchmany(5)   # etracting top 5 results
print(result)

[('B1', 'Division 1A', 'Belgium'), ('D1', 'Bundesliga', 'Deutschland'), ('D2', '2. Bundesliga', 'Deutschland'), ('E0', 'Premier League', 'England'), ('E1', 'EFL Championship', 'England')]


# Examples:

## Creating tables

In [14]:
engine = db.create_engine('sqlite:///datacamp.sqlite')

In [15]:
conn = engine.connect()

In [16]:
metadata = db.MetaData()

In [17]:
# we have the file now

In [18]:
Student = db.Table('Student', metadata,
                   db.Column('Id', db.Integer(), primary_key=True),
                   db.Column('Name', db.String(255), nullable=False),
                   db.Column('Major', db.String(255), default="Math"),
                   db.Column('Pass', db.Boolean(), default=True)
                  )

In [20]:
metadata.create_all(engine)   # updated file now

## Adding row of data

In [22]:
query =  db.insert(Student).values(Id=1, Name='Lukasz', Major='English', Pass=True)
Result = conn.execute(query)

### Checking if data is there

In [23]:
conn.execute(Student.select()).fetchall()

[(1, 'Lukasz', 'English', True)]

## Adding multiple records

In [24]:
query =  db.insert(Student)  # Create an insert query for the Student table

# Create a list of multiple rows with column names and values.
values_list = [{'Id':'2', 'Name':'Nisha', 'Major':"Science", 'Pass':False},
              {'Id':'3', 'Name':'Natasha', 'Major':"Math", 'Pass':True},
              {'Id':'4', 'Name':'Ben', 'Major':"English", 'Pass':False}]

# Execute the query with a second argument as values_list. 
Result = conn.execute(query,values_list)

## Checking if data is there

In [26]:
conn.execute(Student.select()).fetchall()

[(1, 'Lukasz', 'English', True),
 (2, 'Nisha', 'Science', False),
 (3, 'Natasha', 'Math', True),
 (4, 'Ben', 'English', False)]

In [27]:
conn.execute(db.select([Student])).fetchall()

[(1, 'Lukasz', 'English', True),
 (2, 'Nisha', 'Science', False),
 (3, 'Natasha', 'Math', True),
 (4, 'Ben', 'English', False)]

# Querying data using SQL syntax

In [29]:
conn.execute("SELECT * FROM Student").fetchall()

[(1, 'Lukasz', 'English', 1),
 (2, 'Nisha', 'Science', 0),
 (3, 'Natasha', 'Math', 1),
 (4, 'Ben', 'English', 0)]

In [30]:
output = conn.execute("SELECT * FROM Student")
print(output.fetchall())

[(1, 'Lukasz', 'English', 1), (2, 'Nisha', 'Science', 0), (3, 'Natasha', 'Math', 1), (4, 'Ben', 'English', 0)]


## Filtering results

In [ ]:
conn.exe